In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import fetch_openml
%matplotlib inline
plt.style.use('bmh')

### Implementation  Notes

Knn is straight forward algortihm. In order to classify a new data point, it finds the k nearest neighbors of that point
and classifies according to the majority label.
<br>
Here are some notes regarding my Implementation:<br>
##### note1:
I'm using the trivial Euclidean distance. That is:
$$ d(x,y) = \sqrt{\sum _{i}{(x_i-y_i)^2}} $$
Which is the Euclidean Norm. <br>
##### note2:
Choosing the k-smallest elements in an array is an famous interesting issue:<br>
1. Trivial: $O(k\cdot n)$<br>
Iterate k times and pick the next minimum element
2. Better: $O(n\cdot log(n))$<br>
Sort the array keeping the original indices and pick the first k.
3. Best: $O(n)$<br>
This is the optimal solution Selection algorithm. Here I use numpy's partition which implements "introselect" algorithm.


In [2]:
class kNNClassifier:
    def __init__(self, n_neighbors):
        self.n_neighbors = n_neighbors
        self.data = np.empty((1,1))
        self.labels = np.empty((1,1))
    
    def fit(self, X, y):
        self.data = X
        self.labels= y
        
    def _predict_one_point(self,point):
        dist = np.linalg.norm(self.data-point,axis=1) # note 1
        k_smallets = np.argpartition(dist, self.n_neighbors)[:self.n_neighbors] #note 2
        label_count = np.unique(self.labels[k_smallets],return_counts=True) 
        return label_count[0][label_count[1].argmax()]
        
    
    def predict(self, X):
        preds = np.zeros((X.shape[0],1))
        for i in range(X.shape[0]):
            preds[i]=self._predict_one_point(X[i])
        return preds.T[0]
    
    def score(self, predictions, true_labels):
        
        return (np.count_nonzero(predictions-true_labels.astype("int"))/predictions.size)
        
        

Here we are Testing its performence on the MNIST dataset while copmaring it to sklearn performence.

### Load Data

In [3]:
mnist = fetch_openml('mnist_784', as_frame=False)
data = mnist['data']
labels = mnist['target']

idx = np.random.RandomState(0).choice(70000, 11000)
train = data[idx[:10000], :].astype(int)
train_labels = labels[idx[:10000]]
test = data[idx[10000:], :].astype(int)
test_labels = labels[idx[10000:]]

## Testing accuracy

In [ ]:
X_train, Y_train = train[:1000],train_labels[:1000]
accuracy_map = {"k":[], "my_classifier": [], "sklearn_classifier": []}
for k in [1,2,5,10,30,60,100]:
    accuracy_map["k"].append(k)
    
    knn_b = kNNClassifier(k)
    knn_b.fit(X_train,Y_train)
    preds_b = knn_b.predict(test)
    score_b = 1-knn_b.score(preds_b,test_labels)
    accuracy_map["my_classifier"].append(score_b)
    
    sklearn_knn = KNeighborsClassifier(n_neighbors=k)
    sklearn_knn.fit(X_train,Y_train)
    sklearn_score = sklearn_knn.score(test,test_labels)
    accuracy_map["sklearn_classifier"].append(sklearn_score)

accuracy_table = pd.DataFrame(accuracy_map)

In [ ]:
accuracy_table

In [ ]:
sns.lineplot(x='k', y="my_classifier", data=accuracy_table)